<a href="https://colab.research.google.com/github/Krithiga-Balakrishnan/RP-24-25J-146/blob/Research-Paper-Recommendation-Krihiga/Semantic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install transformers datasets scikit-learn
!pip install datasets
from datasets import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import files

# Upload the dataset
uploaded = files.upload()  # Select your 'dataset_Abstract.json'

Saving dataset_Abstract.json to dataset_Abstract.json


In [ ]:
import json
from transformers import LongformerTokenizer, LongformerForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

# Load the dataset
with open("dataset_Abstract.json", "r") as f:
    papers = json.load(f)

# Prepare data: Abstracts and Labels
abstracts = [paper["Abstract"] for paper in papers]
labels = [0] * len(abstracts)  # Replace with actual labels for semantic tasks

# Split into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(abstracts, labels, test_size=0.2, random_state=42)

# Load Longformer tokenizer and model
model_name = "allenai/longformer-base-4096"
tokenizer = LongformerTokenizer.from_pretrained(model_name)
model = LongformerForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize the datasets
def tokenize_data(texts, labels, tokenizer, max_len=4096):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len, return_tensors="pt")
    dataset = torch.utils.data.TensorDataset(encodings["input_ids"], encodings["attention_mask"], torch.tensor(labels))
    return dataset

train_dataset = tokenize_data(train_texts, train_labels, tokenizer)
val_dataset = tokenize_data(val_texts, val_labels, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Use the new 'eval_strategy' keyword
    save_strategy="epoch",  # Match the strategy with 'eval_strategy'
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    save_total_limit=2,  # Limit the number of saved checkpoints
    logging_dir="./logs",
    logging_steps=500,
    load_best_model_at_end=True  # Load the best model after training
)

In [ ]:
# Prepare the training and validation datasets
def prepare_dataset(texts, labels, tokenizer, max_len=4096):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len)
    return Dataset.from_dict({"input_ids": encodings["input_ids"],
                              "attention_mask": encodings["attention_mask"],
                              "labels": labels})

# Tokenize and convert datasets
train_dataset = prepare_dataset(train_texts, train_labels, tokenizer)
val_dataset = prepare_dataset(val_texts, val_labels, tokenizer)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


In [ ]:
# Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Initializing global attention on CLS token...
Input ids are automatically padded to be a multiple of `config.attention_window`: 512


Epoch,Training Loss,Validation Loss
1,No log,0.000243
2,No log,0.000124
3,No log,0.000100


TrainOutput(global_step=60, training_loss=0.037878680229187014, metrics={'train_runtime': 98.8602, 'train_samples_per_second': 1.214, 'train_steps_per_second': 0.607, 'total_flos': 23169414270240.0, 'train_loss': 0.037878680229187014, 'epoch': 3.0})

In [ ]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to save the model
final_model_dir  = "/content/Model/v1"

# Save the model and tokenizer
model.save_pretrained(final_model_dir)
tokenizer.save_pretrained(final_model_dir)

print(f"Model saved successfully to {final_model_dir}!")



Model saved successfully to /content/Model/v1!


In [ ]:
from transformers import LongformerModel

# Load the fine-tuned Longformer model
fine_tuned_model = LongformerModel.from_pretrained("/content/Model/v1")

# Function to generate embeddings
def generate_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=4096)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use [CLS] token's embedding
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return cls_embedding

Some weights of LongformerModel were not initialized from the model checkpoint at /content/Model/v1 and are newly initialized: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import json

# Convert embeddings (assumed to be NumPy arrays) to lists
abstract_embeddings_list = {paper_id: embedding.tolist() for paper_id, embedding in abstract_embeddings.items()}

# Save embeddings to a JSON file
with open("abstract_embeddings.json", "w") as f:
    json.dump(abstract_embeddings_list, f, indent=4)

print("Embeddings successfully saved to abstract_embeddings.json!")


Embeddings successfully saved to abstract_embeddings.json!


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load saved embeddings
with open("abstract_embeddings.json", "r") as f:
    stored_embeddings = json.load(f)

In [ ]:
# Function to find the most similar abstract
def find_most_similar(input_text, tokenizer, model, embeddings):
    input_embedding = generate_embedding(input_text, tokenizer, model)
    similarity_scores = {paper_id: cosine_similarity([input_embedding], [embedding])[0][0] for paper_id, embedding in embeddings.items()}
    most_similar = max(similarity_scores, key=similarity_scores.get)
    return most_similar, similarity_scores[most_similar]

# Load saved embeddings
with open("abstract_embeddings.json", "r") as f:
    stored_embeddings = json.load(f)

# Find the most similar abstract
input_text = " It is derived from the Hyper surface classification (HSC) that is a global classification technique based on Jordan Curve Theorem and no mapping from lower space to higher one is required here. The experiments reveal that the algorithm reduces training sets while maintaining high accuracy of unseen vectors."
most_similar_paper, similarity_score = find_most_similar(input_text, tokenizer, fine_tuned_model, stored_embeddings)

print(f"Most similar paper: {most_similar_paper}, Similarity Score: {similarity_score}")

    # Input text for similarity search
#input_text = " It is derived from the Hyper surface classification (HSC) that is a global classification technique based on Jordan Curve Theorem and no mapping from lower space to higher one is required here. The experiments reveal that the algorithm reduces training sets while maintaining high accuracy of unseen vectors."

# Find the most similar abstract
#most_similar_paper, similarity_score = find_most_similar(input_text, tokenizer, fine_tuned_model, stored_embeddings)
#print(f"Most similar paper ID: {most_similar_paper}, Similarity Score: {similarity_score}")



Most similar paper: Paper_002, Similarity Score: 0.99754106802585


In [ ]:
# Function to find the most similar abstract
def find_top_n_similar(input_text, tokenizer, model, embeddings, top_n=3):
    input_embedding = generate_embedding(input_text, tokenizer, model)

    similarity_scores = {
        paper_id: cosine_similarity([input_embedding], [embedding])[0][0]
        for paper_id, embedding in embeddings.items()
    }

    # Sort papers by similarity score
    sorted_papers = sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True)

    # Get top N most similar papers
    top_papers = sorted_papers[:top_n]

    return top_papers


# Input text for similarity search
input_text = "It is derived from the Hyper surface classification (HSC) that is a global classification technique based on Jordan Curve Theorem and no mapping from lower space to higher one is required here. The experiments reveal that the algorithm reduces training sets while maintaining high accuracy of unseen vectors."

# Find the top N similar abstracts
top_similar_papers = find_top_n_similar(input_text, tokenizer, fine_tuned_model, stored_embeddings, top_n=5)

for paper_id, score in top_similar_papers:
    print(f"Paper ID: {paper_id}, Similarity Score: {score}")

# Debugging: Check if the list is empty and print the length
print(f"Number of similar papers found: {len(top_similar_papers)}")

# Check if the list is empty
if top_similar_papers:
    # Print the top similar papers
    for paper_id, score in top_similar_papers:
        print(f"Paper ID: {paper_id}, Similarity Score: {score}")

    # Print the most similar paper's details
    most_similar_paper = top_similar_papers[0]  # Get the first paper (most similar)
    print(f"Most similar paper ID: {most_similar_paper[0]}, Similarity Score: {most_similar_paper[1]}")
else:
    print("No similar papers found.")


Paper ID: Paper_002, Similarity Score: 0.9976292900648763
Paper ID: Paper_033, Similarity Score: 0.9940608549933904
Paper ID: Paper_024, Similarity Score: 0.9932051626712424
Paper ID: Paper_018, Similarity Score: 0.992995928137151
Paper ID: Paper_044, Similarity Score: 0.9927697788776328
Number of similar papers found: 5
Paper ID: Paper_002, Similarity Score: 0.9976292900648763
Paper ID: Paper_033, Similarity Score: 0.9940608549933904
Paper ID: Paper_024, Similarity Score: 0.9932051626712424
Paper ID: Paper_018, Similarity Score: 0.992995928137151
Paper ID: Paper_044, Similarity Score: 0.9927697788776328
Most similar paper ID: Paper_002, Similarity Score: 0.9976292900648763


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the path to save the model on Google Drive
save_path = "/content/drive/My Drive/Trained_model/semantic_analysis_v2"

# Save the trained model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved to {save_path}")


Model saved to /content/drive/My Drive/Trained_model/semantic_analysis_v2
